In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

# 1. Load và chia nhỏ văn bản
loader = TextLoader("KH001_Don_Xin_Vay_Von.txt", encoding="utf-8")
docs = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " "]
).split_documents(loader.load())

# 2. Khởi tạo embedding model: paraphrase-multilingual-MiniLM-L12-v2
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={"device": "cpu"}  # hoặc "cuda"
)

# 3. Kết nối tới Qdrant
client = QdrantClient(host="localhost", port=6333)

# 4. Xóa và tạo lại collection với đúng dimension (384) & distance
client.recreate_collection(
    collection_name="viet_kb",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)

# 5. Khởi tạo Qdrant vectorstore
qdrant = Qdrant(
    client=client,
    collection_name="viet_kb",
    embeddings=embedding_model
)

# 6. Thêm documents vào vectordb
qdrant.add_documents(documents=docs, batch_size=64)

print("✅ Đã lưu embedding Vietnamese KB vào Qdrant với mô hình MiniLM!")


✅ Đã lưu embedding Vietnamese KB vào Qdrant với mô hình MiniLM!


In [2]:
import os
import glob
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from tqdm import tqdm # Thư viện để hiển thị thanh tiến trình đẹp mắt

# --- 1. Cấu hình ---
DATA_DIRECTORY = "data"
COLLECTION_NAME = "viet_kb" # Đặt tên mới để không bị ghi đè collection cũ
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
CHUNK_SIZE = 1200 # Tăng kích thước chunk để chứa nhiều ngữ cảnh hơn
CHUNK_OVERLAP = 200 # Tăng độ chồng lấn để tránh cắt câu

# --- 2. Tải tất cả các file từ thư mục ---
print(f"🔍 Đang tìm và tải các file từ thư mục '{DATA_DIRECTORY}'...")
all_documents = []
# Tìm tất cả các file có đuôi .txt trong thư mục data
file_paths = glob.glob(os.path.join(DATA_DIRECTORY, "*.txt"))

if not file_paths:
    print(f"❌ Không tìm thấy file .txt nào trong thư mục '{DATA_DIRECTORY}'. Vui lòng kiểm tra lại.")
else:
    for file_path in tqdm(file_paths, desc="Đang tải các file"):
        try:
            loader = TextLoader(file_path, encoding="utf-8")
            all_documents.extend(loader.load())
        except Exception as e:
            print(f"Lỗi khi tải file {file_path}: {e}")
    
    print(f"✅ Đã tải thành công {len(all_documents)} document từ {len(file_paths)} file.")

    # --- 3. Chia nhỏ toàn bộ văn bản ---
    print(f"🔄 Đang chia nhỏ văn bản với chunk_size={CHUNK_SIZE} và chunk_overlap={CHUNK_OVERLAP}...")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        separators=["\n\n", "\n", "---", ".", " "] # Thêm '---' để có thể tách các file
    )
    docs_for_db = text_splitter.split_documents(all_documents)
    print(f"✅ Đã chia thành {len(docs_for_db)} chunk để chuẩn bị embedding.")

    # --- 4. Khởi tạo embedding model ---
    print(f"🧠 Đang khởi tạo embedding model: {MODEL_NAME}...")
    embedding_model = HuggingFaceEmbeddings(
        model_name=MODEL_NAME,
        model_kwargs={"device": "cpu"}  # Đổi thành "cuda" nếu bạn có GPU
    )
    # Lấy dimension của vector từ model. MiniLM là 384.
    embed_dim = embedding_model.client.get_sentence_embedding_dimension()
    print(f"✅ Model đã sẵn sàng. Kích thước vector: {embed_dim}")

    # --- 5. Kết nối và tạo lại collection trong Qdrant ---
    print("☁️ Đang kết nối tới Qdrant và tạo collection...")
    client = QdrantClient(host="localhost", port=6333)

    # Xóa collection cũ (nếu có) và tạo lại với cấu hình mới
    client.recreate_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(size=embed_dim, distance=Distance.COSINE)
    )
    print(f"✅ Collection '{COLLECTION_NAME}' đã được tạo thành công.")

    # --- 6. Thêm các chunk đã xử lý vào Qdrant ---
    print("⏳ Đang thực hiện embedding và lưu vào Qdrant. Quá trình này có thể mất vài phút...")
    # LangChain Qdrant sẽ tự động xử lý việc gọi embedding_model
    qdrant = Qdrant(
        client=client,
        collection_name=COLLECTION_NAME,
        embeddings=embedding_model
    )

    qdrant.add_documents(documents=docs_for_db, batch_size=64)

    print("\n🎉 HOÀN TẤT! Toàn bộ dữ liệu đã được embedding và lưu vào Qdrant.")
    print(f"👉 Bây giờ bạn có thể sử dụng collection '{COLLECTION_NAME}' trong Langflow.")

🔍 Đang tìm và tải các file từ thư mục 'data'...


Đang tải các file: 100%|██████████| 7/7 [00:00<00:00, 329.18it/s]

✅ Đã tải thành công 7 document từ 7 file.
🔄 Đang chia nhỏ văn bản với chunk_size=1200 và chunk_overlap=200...
✅ Đã chia thành 25 chunk để chuẩn bị embedding.
🧠 Đang khởi tạo embedding model: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2...


✅ Model đã sẵn sàng. Kích thước vector: 384
☁️ Đang kết nối tới Qdrant và tạo collection...


/tmp/ipykernel_8208/2750828601.py:61: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


✅ Collection 'viet_kb' đã được tạo thành công.
⏳ Đang thực hiện embedding và lưu vào Qdrant. Quá trình này có thể mất vài phút...


/tmp/ipykernel_8208/2750828601.py:70: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  qdrant = Qdrant(



🎉 HOÀN TẤT! Toàn bộ dữ liệu đã được embedding và lưu vào Qdrant.
👉 Bây giờ bạn có thể sử dụng collection 'viet_kb' trong Langflow.


In [39]:
import requests
import json
import time
from pprint import pprint

# --- 1. CẤU HÌNH ---

# URL API của pipeline Langflow
LANGFLOW_URL = "http://127.0.0.1:7860/api/v1/run/f9835000-2598-4144-b166-b941759e59b4"

# Headers cho request, chỉ định rõ encoding UTF-8
HEADERS = {"Content-Type": "application/json; charset=utf-8"}

# Các tham số cho logic lặp
MAX_ITERATIONS = 10      # Số lần lặp tối đa để tránh bị kẹt
QUESTIONS_PER_BATCH = 7 # Số trường thông tin hỏi trong mỗi lần gọi API

# Danh sách các trường thông tin cần trích xuất (hoàn toàn bằng tiếng Việt)
# Đây vừa là mục tiêu, vừa là key trong JSON cuối cùng.
FIELDS_TO_EXTRACT = [
    # ==== Phần 1: Thông tin pháp lý của khách hàng (Từ Giấy ĐKKD, Đơn vay) ====
    "tên đầy đủ của doanh nghiệp",
    "mã số doanh nghiệp",
    "ngày cấp giấy chứng nhận đăng ký kinh doanh",
    "nơi cấp giấy chứng nhận đăng ký kinh doanh",
    "địa chỉ trụ sở chính",
    "ngành nghề kinh doanh chính",
    "vốn điều lệ",
    "tên người đại diện theo pháp luật",
    "chức vụ người đại diện",
    "số CCCD/CMND của người đại diện",
    
    # ==== Phần 2: Thông tin về khoản vay đề xuất (Từ Đơn vay, Hợp đồng tín dụng) ====
    "số tiền đề nghị vay",
    "thời hạn vay (tháng)",
    "mục đích vay vốn",
    "sản phẩm tín dụng đăng ký",
    "lãi suất cho vay (%/năm)",
    "phí trả nợ trước hạn (%)",
    "phương thức trả nợ",
    
    # ==== Phần 3: Tình hình tài chính và kinh doanh (Từ Đơn vay, Phương án KD) ====
    "doanh thu năm gần nhất",
    "lợi nhuận ròng năm gần nhất",
    "tổng tài sản ước tính",
    "dự báo doanh thu năm 1 sau khi vay",
    "kế hoạch phân bổ vốn vay cho đầu tư kho lạnh",
    
    # ==== Phần 4: Lịch sử và quan hệ tín dụng (Từ Báo cáo CIC, Đơn vay) ====
    "điểm tín dụng doanh nghiệp (CIC)",
    "xếp hạng tín dụng doanh nghiệp (CIC)",
    "phân loại nợ hiện tại của doanh nghiệp (CIC)",
    "tổng dư nợ tại các tổ chức tín dụng khác",
    "dư nợ tại Vietcombank (CIC)",
    "lịch sử trả nợ 12 tháng gần nhất (chuỗi trạng thái CIC)",
    
    # ==== Phần 5: Thông tin về tài sản bảo đảm (Từ Hồ sơ TSĐB, Hợp đồng tín dụng) ====
    "loại tài sản bảo đảm chính",
    "chủ sở hữu tài sản bảo đảm",
    "giá trị tài sản bảo đảm theo thẩm định",
    "đơn vị thẩm định giá",
    "tỷ lệ cho vay tối đa trên tài sản bảo đảm (%)",
    "tên người bảo lãnh cho khoản vay"
]


# --- 2. HÀM GỬI YÊU CẦU VÀ XỬ LÝ PHẢN HỒI ---

def query_langflow_for_json(question_prompt: str) -> dict:
    """
    Gửi một prompt đến Langflow, nhận phản hồi và cố gắng trích xuất một đối tượng JSON.
    """
    # Dữ liệu gửi đi, prompt tiếng Việt được đặt làm giá trị
    payload = {
        "input_value": question_prompt,
        "output_type": "chat",
        "input_type": "chat"
    }

    try:
        # requests sẽ tự động mã hóa payload thành chuỗi JSON hợp lệ (bao gồm cả ký tự Unicode)
        response = requests.post(LANGFLOW_URL, json=payload, headers=HEADERS, timeout=120)
        response.raise_for_status() # Báo lỗi nếu status code là 4xx hoặc 5xx

        # response.text sẽ trả về chuỗi đã được giải mã UTF-8
        # json.loads sẽ parse chuỗi JSON của Langflow để lấy ra đối tượng Python
        langflow_data = json.loads(response.text)
        
        # Đường dẫn để lấy text trả về từ LLM
        llm_response_text = langflow_data['outputs'][0]['outputs'][0]['results']['message']['text']

        # Tìm và parse khối JSON bên trong phản hồi của LLM
        # Đây là bước quan trọng để xử lý các trường hợp LLM trả về thêm văn bản thừa
        start = llm_response_text.find('{')
        end = llm_response_text.rfind('}')
        if start != -1 and end != -1:
            json_str = llm_response_text[start : end + 1]
            return json.loads(json_str)
        else:
            print("  - Lỗi: Không tìm thấy đối tượng JSON hợp lệ trong phản hồi của LLM.")
            return {}

    except requests.exceptions.RequestException as e:
        print(f"  - Lỗi kết nối API: {e}")
        return {}
    except (KeyError, IndexError):
        print("  - Lỗi: Cấu trúc JSON trả về từ Langflow không như mong đợi.")
        pprint(langflow_data) # In ra để debug
        return {}
    except json.JSONDecodeError as e:
        print(f"  - Lỗi: Không thể phân tích JSON từ phản hồi của LLM. {e}")
        print(f"  - Phản hồi gốc từ LLM: {llm_response_text}")
        return {}


# --- 3. LOGIC TRÍCH XUẤT LẶP ---

def main():
    """
    Hàm chính điều khiển luồng trích xuất thông tin lặp đi lặp lại.
    """
    final_result = {}
    remaining_fields = FIELDS_TO_EXTRACT.copy()
    current_iteration = 0

    print("🚀 Bắt đầu quá trình trích xuất thông tin tự động...")

    while remaining_fields and current_iteration < MAX_ITERATIONS:
        current_iteration += 1
        print(f"\n--- VÒNG LẶP {current_iteration}/{MAX_ITERATIONS} ---")
        print(f"Trường thông tin còn lại: {len(remaining_fields)}")

        # Lấy một lô các trường để hỏi trong lần lặp này
        batch_fields = remaining_fields[:QUESTIONS_PER_BATCH]
        
        # Tạo danh sách các trường dưới dạng một chuỗi text thuần túy, dễ đọc
        fields_as_text_list = "\n- ".join(batch_fields)
        
        # Xây dựng một prompt chất lượng cao, rõ ràng và giàu ngữ nghĩa
        prompt = f"""
- {fields_as_text_list}
"""
        
        print(f"Đang gửi yêu cầu cho các trường: {batch_fields}")
        
        # Gửi prompt và nhận về kết quả đã được parse
        response_json = query_langflow_for_json(prompt)

        if not response_json:
            print("  - Không nhận được phản hồi hợp lệ. Chuyển sang vòng lặp tiếp theo.")
            time.sleep(2)
            continue
        
        newly_answered_fields = []
        for field, value in response_json.items():
            # Kiểm tra xem trường này có phải là cái ta đang tìm không, và giá trị có hợp lệ không
            if field in remaining_fields and value is not None and str(value).strip() != "":
                print(f"    ✅ Đã tìm thấy '{field}': {value}")
                final_result[field] = value
                newly_answered_fields.append(field)

        # Cập nhật lại danh sách các trường còn lại
        if newly_answered_fields:
            remaining_fields = [f for f in remaining_fields if f not in newly_answered_fields]
        else:
            print("  - Không có thông tin mới nào được tìm thấy trong vòng lặp này.")
        
        time.sleep(1) # Tạm dừng giữa các lần gọi API

    # --- 4. TỔNG KẾT ---
    print("\n\n✅ Quá trình trích xuất hoàn tất!")
    if remaining_fields:
        print(f"⚠️ Không thể trích xuất các trường sau: {remaining_fields}")

    print("-----------------------------------------")
    print("          KẾT QUẢ JSON CUỐI CÙNG         ")
    print("-----------------------------------------")
    
    # In kết quả JSON cuối cùng, đảm bảo hiển thị đúng tiếng Việt
    print(json.dumps(final_result, indent=4, ensure_ascii=False))


if __name__ == "__main__":
    main()

🚀 Bắt đầu quá trình trích xuất thông tin tự động...

--- VÒNG LẶP 1/10 ---
Trường thông tin còn lại: 34
Đang gửi yêu cầu cho các trường: ['tên đầy đủ của doanh nghiệp', 'mã số doanh nghiệp', 'ngày cấp giấy chứng nhận đăng ký kinh doanh', 'nơi cấp giấy chứng nhận đăng ký kinh doanh', 'địa chỉ trụ sở chính', 'ngành nghề kinh doanh chính', 'vốn điều lệ']
    ✅ Đã tìm thấy 'tên đầy đủ của doanh nghiệp': Đặng và Lê Công ty TNHH MTV Cổ phần
    ✅ Đã tìm thấy 'mã số doanh nghiệp': 2532514855
    ✅ Đã tìm thấy 'địa chỉ trụ sở chính': 521 Đặng Tổ, JohnXã, 435086

--- VÒNG LẶP 2/10 ---
Trường thông tin còn lại: 31
Đang gửi yêu cầu cho các trường: ['ngày cấp giấy chứng nhận đăng ký kinh doanh', 'nơi cấp giấy chứng nhận đăng ký kinh doanh', 'ngành nghề kinh doanh chính', 'vốn điều lệ', 'tên người đại diện theo pháp luật', 'chức vụ người đại diện', 'số CCCD/CMND của người đại diện']
    ✅ Đã tìm thấy 'vốn điều lệ': 13,509,534,123 VNĐ
    ✅ Đã tìm thấy 'tên người đại diện theo pháp luật': Bác Tú Lê


In [4]:
import requests
import json
import time
from pprint import pprint

# --- 1. CẤU HÌNH ---

# URL API của pipeline Langflow
LANGFLOW_URL = "http://127.0.0.1:7860/api/v1/run/f9835000-2598-4144-b166-b941759e59b4"

# Headers cho request, chỉ định rõ encoding UTF-8
HEADERS = {"Content-Type": "application/json; charset=utf-8"}

# Các tham số cho logic lặp
MAX_ITERATIONS = 20      # Tăng số lần lặp tối đa vì có thể phải chia nhỏ nhiều lần
INITIAL_BATCH_SIZE = 6 # Số trường thông tin hỏi trong lô ban đầu
MIN_BATCH_SIZE_TO_SPLIT = 2 # Chỉ chia nhỏ lô nếu nó còn nhiều hơn 1 trường

# Danh sách các trường thông tin cần trích xuất (hoàn toàn bằng tiếng Việt)
FIELDS_TO_EXTRACT = [
    # ==== Phần 1: Thông tin pháp lý của khách hàng (Từ Giấy ĐKKD, Đơn vay) ====
    "tên đầy đủ của doanh nghiệp",
    "mã số doanh nghiệp",
    "ngày cấp giấy chứng nhận đăng ký kinh doanh",
    "nơi cấp giấy chứng nhận đăng ký kinh doanh",
    "địa chỉ trụ sở chính",
    "ngành nghề kinh doanh chính",
    "vốn điều lệ",
    "tên người đại diện theo pháp luật",
    "chức vụ người đại diện",
    "số CCCD/CMND của người đại diện",
    
    # ==== Phần 2: Thông tin về khoản vay đề xuất (Từ Đơn vay, Hợp đồng tín dụng) ====
    "số tiền đề nghị vay",
    "thời hạn vay (tháng)",
    "mục đích vay vốn",
    "sản phẩm tín dụng đăng ký",
    "lãi suất cho vay (%/năm)",
    "phí trả nợ trước hạn (%)",
    "phương thức trả nợ",
    
    # ==== Phần 3: Tình hình tài chính và kinh doanh (Từ Đơn vay, Phương án KD) ====
    "doanh thu năm gần nhất",
    "lợi nhuận ròng năm gần nhất",
    "tổng tài sản ước tính",
    "dự báo doanh thu năm 1 sau khi vay",
    "kế hoạch phân bổ vốn vay cho đầu tư kho lạnh",
    
    # ==== Phần 4: Lịch sử và quan hệ tín dụng (Từ Báo cáo CIC, Đơn vay) ====
    "điểm tín dụng doanh nghiệp (CIC)",
    "xếp hạng tín dụng doanh nghiệp (CIC)",
    "phân loại nợ hiện tại của doanh nghiệp (CIC)",
    "tổng dư nợ tại các tổ chức tín dụng khác",
    "dư nợ tại Vietcombank (CIC)",
    "lịch sử trả nợ 12 tháng gần nhất (chuỗi trạng thái CIC)",
    
    # ==== Phần 5: Thông tin về tài sản bảo đảm (Từ Hồ sơ TSĐB, Hợp đồng tín dụng) ====
    "loại tài sản bảo đảm chính",
    "chủ sở hữu tài sản bảo đảm",
    "giá trị tài sản bảo đảm theo thẩm định",
    "đơn vị thẩm định giá",
    "tỷ lệ cho vay tối đa trên tài sản bảo đảm (%)",
    "tên người bảo lãnh cho khoản vay"
]


# --- 2. CÁC HÀM HỖ TRỢ ---

def create_prompt(fields_list: list) -> str:
    """Tạo prompt động dựa trên số lượng trường cần hỏi."""
    if not fields_list:
        return ""
    
    # Tạo danh sách các trường dưới dạng một chuỗi text thuần túy
    fields_as_text_list = "\n- ".join(fields_list)
    
    # Thay đổi câu lệnh prompt để rõ ràng hơn với LLM
    if len(fields_list) == 1:
        # Prompt tập trung vào một trường duy nhất
        return f"{fields_as_text_list}"
    else:
        # Prompt cho một danh sách các trường
        return f"""
- {fields_as_text_list}
"""

def is_valid_value(value) -> bool:
    """Kiểm tra xem giá trị trả về có hợp lệ không (không phải None, không rỗng)."""
    return value is not None and str(value).strip() != ""

def query_langflow_for_json(question_prompt: str) -> dict:
    """
    Gửi một prompt đến Langflow, nhận phản hồi và cố gắng trích xuất một đối tượng JSON.
    (Hàm này giữ nguyên như cũ, không cần thay đổi)
    """
    if not question_prompt:
        return {}
    payload = {
        "input_value": question_prompt, "output_type": "chat", "input_type": "chat"
    }
    try:
        response = requests.post(LANGFLOW_URL, json=payload, headers=HEADERS, timeout=120)
        response.raise_for_status()
        langflow_data = json.loads(response.text)
        llm_response_text = langflow_data['outputs'][0]['outputs'][0]['results']['message']['text']
        start = llm_response_text.find('{')
        end = llm_response_text.rfind('}')
        if start != -1 and end != -1:
            json_str = llm_response_text[start : end + 1]
            return json.loads(json_str)
        else:
            print("  - Lỗi: Không tìm thấy đối tượng JSON hợp lệ trong phản hồi của LLM.")
            return {}
    except requests.exceptions.RequestException as e:
        print(f"  - Lỗi kết nối API: {e}")
        return {}
    except (KeyError, IndexError):
        print("  - Lỗi: Cấu trúc JSON trả về từ Langflow không như mong đợi.")
        pprint(langflow_data)
        return {}
    except json.JSONDecodeError as e:
        print(f"  - Lỗi: Không thể phân tích JSON từ phản hồi của LLM. {e}")
        print(f"  - Phản hồi gốc từ LLM: {llm_response_text}")
        return {}


# --- 3. LOGIC TRÍCH XUẤT "CHIA ĐỂ TRỊ" ---

def main():
    """
    Hàm chính điều khiển luồng trích xuất thông tin thông minh.
    Sử dụng một hàng đợi (queue) để quản lý các lô công việc.
    Nếu một lô thất bại, nó sẽ được chia nhỏ và thêm lại vào hàng đợi.
    """
    final_result = {}
    
    # Khởi tạo hàng đợi công việc (work queue)
    # Ban đầu, chia toàn bộ danh sách thành các lô có kích thước `INITIAL_BATCH_SIZE`
    work_queue = [
        FIELDS_TO_EXTRACT[i:i + INITIAL_BATCH_SIZE]
        for i in range(0, len(FIELDS_TO_EXTRACT), INITIAL_BATCH_SIZE)
    ]
    
    current_iteration = 0
    print("🚀 Bắt đầu quá trình trích xuất thông tin với logic 'Chia để trị'...")

    # Vòng lặp sẽ tiếp tục khi vẫn còn việc trong hàng đợi và chưa vượt quá giới hạn
    while work_queue and current_iteration < MAX_ITERATIONS:
        current_iteration += 1
        
        # Lấy lô công việc tiếp theo từ đầu hàng đợi
        current_batch = work_queue.pop(0)
        
        print(f"\n--- VÒNG LẶP {current_iteration}/{MAX_ITERATIONS} | Lô còn lại: {len(work_queue)} ---")
        print(f"Đang xử lý lô gồm {len(current_batch)} trường: {current_batch}")
        
        # Tạo prompt và gửi yêu cầu
        prompt = create_prompt(current_batch)
        response_json = query_langflow_for_json(prompt)

        newly_found_fields = []
        if response_json:
            for field in current_batch:
                # Kiểm tra xem trường có trong phản hồi và có giá trị hợp lệ không
                if field in response_json and is_valid_value(response_json[field]):
                    value = response_json[field]
                    print(f"    ✅ Đã tìm thấy '{field}': {value}")
                    final_result[field] = value
                    newly_found_fields.append(field)
        
        # *** LOGIC "CHIA ĐỂ TRỊ" NẰM Ở ĐÂY ***
        
        # Xác định các trường chưa tìm được trong lô này
        failed_fields = [f for f in current_batch if f not in newly_found_fields]
        
        if failed_fields:
            print(f"  - Không tìm thấy {len(failed_fields)} trường: {failed_fields}")
            
            # Nếu lô thất bại còn đủ lớn để chia, thì chia đôi nó ra
            if len(failed_fields) >= MIN_BATCH_SIZE_TO_SPLIT:
                print(f"  -> splitting_batch Chia nhỏ lô thất bại và thêm lại vào hàng đợi.")
                mid_point = len(failed_fields) // 2
                first_half = failed_fields[:mid_point]
                second_half = failed_fields[mid_point:]
                
                # Thêm 2 lô nhỏ hơn vào ĐẦU hàng đợi để được ưu tiên xử lý ngay
                work_queue.insert(0, second_half)
                work_queue.insert(0, first_half)
            else:
                # Nếu lô quá nhỏ để chia (chỉ còn 1 trường), ta coi như thất bại cuối cùng
                print(f"  -> Lô quá nhỏ, không chia nữa. Ghi nhận là không tìm thấy.")
                # Chúng ta sẽ báo cáo các trường này ở cuối
        
        time.sleep(3) # Tạm dừng giữa các lần gọi API

    # --- 4. TỔNG KẾT ---
    print("\n\n✅ Quá trình trích xuất hoàn tất!")
    
    # Các trường còn lại trong hàng đợi hoặc không thể tìm thấy sau khi chia nhỏ
    remaining_fields = set(FIELDS_TO_EXTRACT) - set(final_result.keys())

    if remaining_fields:
        print(f"⚠️ Không thể trích xuất {len(remaining_fields)} trường sau:")
        pprint(list(remaining_fields))

    print("-----------------------------------------")
    print("          KẾT QUẢ JSON CUỐI CÙNG         ")
    print("-----------------------------------------")
    
    print(json.dumps(final_result, indent=4, ensure_ascii=False))


if __name__ == "__main__":
    main()
    

🚀 Bắt đầu quá trình trích xuất thông tin với logic 'Chia để trị'...

--- VÒNG LẶP 1/20 | Lô còn lại: 5 ---
Đang xử lý lô gồm 6 trường: ['tên đầy đủ của doanh nghiệp', 'mã số doanh nghiệp', 'ngày cấp giấy chứng nhận đăng ký kinh doanh', 'nơi cấp giấy chứng nhận đăng ký kinh doanh', 'địa chỉ trụ sở chính', 'ngành nghề kinh doanh chính']
    ✅ Đã tìm thấy 'tên đầy đủ của doanh nghiệp': Đặng và Lê Công ty TNHH MTV Cổ phần
    ✅ Đã tìm thấy 'mã số doanh nghiệp': 2532514855
    ✅ Đã tìm thấy 'ngày cấp giấy chứng nhận đăng ký kinh doanh': null
    ✅ Đã tìm thấy 'nơi cấp giấy chứng nhận đăng ký kinh doanh': null
    ✅ Đã tìm thấy 'địa chỉ trụ sở chính': 521 Đặng Tổ, JohnXã, 435086
    ✅ Đã tìm thấy 'ngành nghề kinh doanh chính': Sản xuất thực phẩm

--- VÒNG LẶP 2/20 | Lô còn lại: 4 ---
Đang xử lý lô gồm 6 trường: ['vốn điều lệ', 'tên người đại diện theo pháp luật', 'chức vụ người đại diện', 'số CCCD/CMND của người đại diện', 'số tiền đề nghị vay', 'thời hạn vay (tháng)']
    ✅ Đã tìm thấy 'tên

In [2]:
import requests
from pprint import pprint
url = "http://127.0.0.1:7860/api/v1/run/b0f30b0e-ff82-4248-916e-d44482874147"  # The complete API endpoint URL for this flow

# Request payload configuration
payload = {
    "input_value": "số tiền đề nghị vay",  # The input value to be processed by the flow
    "output_type": "chat",  # Specifies the expected output format
    "input_type": "chat"  # Specifies the input format
}

# Request headers
headers = {
    "Content-Type": "application/json"
}

try:
    # Send API request
    response = requests.request("POST", url, json=payload, headers=headers)
    response.raise_for_status()  # Raise exception for bad status codes

    # Print response
    pprint(response.text)

except requests.exceptions.RequestException as e:
    print(f"Error making API request: {e}")
except ValueError as e:
    print(f"Error parsing response: {e}")


('{"session_id":"b0f30b0e-ff82-4248-916e-d44482874147","outputs":[{"inputs":{"input_value":"số '
 'tiền đề nghị '
 'vay"},"outputs":[{"results":{"message":{"text_key":"text","data":{"timestamp":"2025-07-10 '
 '02:39:42 '
 'UTC","sender":"Machine","sender_name":"AI","session_id":"b0f30b0e-ff82-4248-916e-d44482874147","text":"Dựa '
 'trên thông tin được cung cấp từ tài liệu \\"KH001_Hop_Dong_Tin_Dung.txt\\", '
 'số tiền đề nghị vay là 8,972,916,744 VNĐ (Bằng chữ: Đang để vài hớn. Nào và '
 'hớn không là. Của các hớn nhưng về. đồng). Khoản vay này được dùng cho mục '
 'đích đầu tư kho '
 'lạnh.\\n","files":[],"error":false,"edit":false,"properties":{"text_color":"","background_color":"","edited":false,"source":{"id":"GoogleGenerativeAIModel-59i2F","display_name":"Google '
 'Generative '
 'AI","source":"gemini-2.0-flash"},"icon":"GoogleGenerativeAI","allow_markdown":false,"positive_feedback":null,"state":"complete","targets":[]},"category":"message","content_blocks":[],"id":"3fd358ac-c410-